In [1]:
import numpy as np
import math
import hashlib
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from collections import defaultdict

In [8]:
class NaiveBayes:
    #__tamanhoBroaCast= 0
    __treinoRDD = None
    __PARDD  = None
    __dictPx = {}
    
    def __init__(self):
        return
    
    def Treino(self, treinoRDD):
        self.__treinoRDD = treinoRDD
        #self.__tamanhoBroaCast =  sc.broadcast(treinoRDD.count())
        self.__PreCalculaPA(treinoRDD)
        self.__CalculaPx(treinoRDD)
        return
    
    def __PreCalculaPA(self,treinoRDD):
        N = treinoRDD.count()
        self.__PARDD = (treinoRDD
                        .flatMap(lambda x: [((y,x.features[y]),1) for y in x.features.indices])
                        .reduceByKey(lambda x,y:x+y)
                        .map(lambda x:(x[0],math.log(x[1]/float(N))))
                          )
        return
    
    def __CalculaPx(self,treinoRDD):
        N = treinoRDD.count()
        self.__dictPx = (treinoRDD
                                .map(lambda x: (x.label,1))
                                .reduceByKey(lambda x,y:x+y)
                                .map(lambda x: (x[0], math.log(x[1]/float(N))))
                                .collectAsMap()
                           )
        return
    
    def Predicao(self, objetoDesconhecido, mostraValores = False):
        maxClassificao = (-1,-100000000000000)
        for key in self.__dictPx.keys():
                PAix = self.__CalculaPAix(objetoDesconhecido,key,self.__treinoRDD)
                PAi = self.__CalculaPAi(objetoDesconhecido,self.__PARDD)
                Px = self.__dictPx[key]
                PxAi = PAix + Px - PAi
                
                if mostraValores:
                    print 'Key:' + str(key)
                    print '\t Px: ' + str(Px)
                    print '\t PAix: ' + str(PAix)
                    print '\t PAi: ' + str(PAi)
                    print '\t Conta(Log(P(Ai|x))+ Log(P(x)) - Log(P(Ai))): (Log(' + str(Px) + ') + Log('+ str(PAix) + ') - Log(' + str(PxAi) + ')'
                    print '\t Resultado: (Log(P(Ai|x))+ Log(P(x)) - Log(P(Ai))): ' + str(PxAi)
                if maxClassificao[1] < PxAi:
                    maxClassificao = (key,PxAi)
                    
        return maxClassificao[0]
    
    def __CalculaPAix(self, objetoDesconhecido, key,treinoRDD):
        retorno = 1
        N = treinoRDD.count()
        treinoFiltroRDD = treinoRDD.filter(lambda x: x.label == key)
        quantidadeTreioFiltro = treinoFiltroRDD.count()        
        
        for i in (objetoDesconhecido.features.indices):            
            quantidadePorFeature = (treinoFiltroRDD
                                     .filter(lambda x: i in x.features.indices)
                                     .count()
                                   )
            retorno += math.log((1 + quantidadePorFeature)/float(quantidadeTreioFiltro + N))
        return retorno
    
    def __CalculaPAi(self, objetoDesconhecido,PARDD):
        retorno = 1
        for i in (objetoDesconhecido.features.indices):
            retorno += (PARDD
                        .filter(lambda x: x[0][0] == i and x[0][1] == objetoDesconhecido.features[i])
                        .take(1)[0][1]
                       )
        return retorno

In [9]:
def parsePoint(point):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
    pPoint = point.split(',')[1:]
    return [(pId, pValue) for (pId, pValue) in enumerate(pPoint)]

def createOneHotDict(inputData):
    """Creates a one-hot-encoder dictionary based on the input data.

    Args:
        inputData (RDD of lists of (int, str)): An RDD of observations where each observation is
            made up of a list of (featureID, value) tuples.

    Returns:
        dict: A dictionary where the keys are (featureID, value) tuples and map to values that are
            unique integers.
    """
    
    return (inputData
            .flatMap(lambda sd: sd)
            .distinct()
            .zipWithIndex()
            .collectAsMap()
           )

def oneHotEncoding(rawFeats, OHEDict, numOHEFeats):
    """Produce a one-hot-encoding from a list of features and an OHE dictionary.

    Note:
        If a (featureID, value) tuple doesn't have a corresponding key in OHEDict it should be
        ignored.

    Args:
        rawFeats (list of (int, str)): The features corresponding to a single observation.  Each
            feature consists of a tuple of featureID and the feature's value. (e.g. sampleOne)
        OHEDict (dict): A mapping of (featureID, value) to unique integer.
        numOHEFeats (int): The total number of unique OHE features (combinations of featureID and
            value).LabelAndFeatures

    Returns:
        SparseVector: A SparseVector of length numOHEFeats with indicies equal to the unique
            identifiers for the (featureID, value) combinations that occur in the observation and
            with values equal to 1.0.
    """
    return SparseVector(numOHEFeats,[(OHEDict[x],1) for x in rawFeats if OHEDict.has_key(x)])

def parseOHEPoint(point, OHEDict, numOHEFeats):
    """Obtain the label and feature vector for this raw observation.

    Note:
        You must use the function `oneHotEncoding` in this implementation or later portions
        of this lab may not function as expected.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.
        OHEDict (dict of (int, str) to int): Mapping of (featureID, value) to unique integer.
        numOHEFeats (int): The number of unique features in the training dataset.

    Returns:
        LabeledPoint: Contains the label for the observation and the one-hot-encoding of the
            raw features based on the provided OHE dictionary.
    """
    pPoint = point.split(',')
    return LabeledPoint(pPoint[0], oneHotEncoding(parsePoint(point), OHEDict, numOHEFeats))

def hashFunction(numBuckets, rawFeats, printMapping=False):
    """Calculate a feature dictionary for an observation's features based on hashing.

    Note:
        Use printMapping=True for debug purposes and to better understand how the hashing works.

    Args:
        numBuckets (int): Number of buckets to use as features.
        rawFeats (list of (int, str)): A list of features for an observation.  Represented as
            (featureID, value) tuples.
        printMapping (bool, optional): If true, the mappings of featureString to index will be
            printed.

    Returns:
        dict of int to float:  The keys will be integers which represent the buckets that the
            features have been hashed to.  The value for a given key will contain the count of the
            (featureID, value) tuples that have hashed to that key.
    """
    mapping = {}
    for featureString in rawFeats.split():
        mapping[featureString] = int(int(hashlib.md5(featureString).hexdigest(), 16) % numBuckets)
    if(printMapping): print mapping
    sparseFeatures = defaultdict(float)
    for bucket in mapping.values():
        sparseFeatures[bucket] += 1.0
    return dict(sparseFeatures)

def parseTextPoint(point, numBuckets):
    #id0, id1, id2, text, sent = point.split('\t') #Exemplo Professor
    id1, id2, text, sent = point.split('\t') #Exemplo Professor
    sent = int(sent)
    if sent < 2:
        label = 0
    elif sent > 2:
        label = 1
    else:
        label = 2
    print text
    features = SparseVector(numBuckets,hashFunction(numBuckets, text))
    
    return LabeledPoint(label,features)

In [4]:
#Seguro 0
#Perigo 1
dictTmp = {}
dictTmp[0] = "seguro"
dictTmp[1] = "Perigo"

listaTreino = []
listaTreino.append("0,peludo,marrom,grande,dura")
listaTreino.append("0,peludo,verde,grande,dura")
listaTreino.append("1,liso,vermelho,grande,macia")
listaTreino.append("0,peludo,verde,grande,macia")
listaTreino.append("0,peludo,vermelho,pequeno,dura")
listaTreino.append("0,liso,vermelho,pequeno,dura")
listaTreino.append("0,liso,marrom,pequeno,dura")
listaTreino.append("1,peludo,verde,pequeno,macia")
listaTreino.append("1,liso,verde,pequeno,dura")
listaTreino.append("0,peludo,vermelho,grande,dura")
listaTreino.append("0,liso,marrom,grande,macia")
listaTreino.append("1,liso,verde,pequeno,macia")
listaTreino.append("0,peludo,vermelho,pequeno,macia")
listaTreino.append("1,liso,vermelho,grande,dura")

sampleDataRDD = sc.parallelize(listaTreino)
ctrOHEDict = createOneHotDict(sampleDataRDD.map(parsePoint))
numCtrOHEFeats = len(ctrOHEDict.keys())
OHETrainData = sampleDataRDD.map(lambda point: parseOHEPoint(point, ctrOHEDict, numCtrOHEFeats))

nb = NaiveBayes()
nb.Treino(OHETrainData)

objetoDesconhecido1 = ("3,liso,vermelho,pequeno,dura")
lpObjetoDesconhecido1 =  parseOHEPoint(objetoDesconhecido1, ctrOHEDict, numCtrOHEFeats)

objetoDesconhecido2 = ("3,peludo,verde,pequeno,dura")
lpObjetoDesconhecido2 = parseOHEPoint(objetoDesconhecido2, ctrOHEDict, numCtrOHEFeats)

print "Classificando objeto 1:"
retorno1 = nb.Predicao(lpObjetoDesconhecido1,True)

print "Classificando objeto 2:"
retorno2 = nb.Predicao(lpObjetoDesconhecido2,True)

print "Resultado1: " + dictTmp[retorno1]
print "Resultado2: " + dictTmp[retorno2]

Classificando objeto 1:
Key:0.0
	 Px: -0.441832752279
	 PAix: -2.12593782921
	 PAi: -1.79320800944
	 Conta(Log(P(Ai|x))+ Log(P(x)) - Log(P(Ai))): (Log(-0.441832752279) + Log(-2.12593782921) - Log(-0.774562572045)
	 Resultado: (Log(P(Ai|x))+ Log(P(x)) - Log(P(Ai))): -0.774562572045
Key:1.0
	 Px: -1.02961941718
	 PAix: -1.56655063883
	 PAi: -1.79320800944
	 Conta(Log(P(Ai|x))+ Log(P(x)) - Log(P(Ai))): (Log(-1.02961941718) + Log(-1.56655063883) - Log(-0.802962046567)
	 Resultado: (Log(P(Ai|x))+ Log(P(x)) - Log(P(Ai))): -0.802962046567
Classificando objeto 2:
Key:0.0
	 Px: -0.441832752279
	 PAix: -2.12593782921
	 PAi: -1.97552956624
	 Conta(Log(P(Ai|x))+ Log(P(x)) - Log(P(Ai))): (Log(-0.441832752279) + Log(-2.12593782921) - Log(-0.592241015251)
	 Resultado: (Log(P(Ai|x))+ Log(P(x)) - Log(P(Ai))): -0.592241015251
Key:1.0
	 Px: -1.02961941718
	 PAix: -2.54737989184
	 PAi: -1.97552956624
	 Conta(Log(P(Ai|x))+ Log(P(x)) - Log(P(Ai))): (Log(-1.02961941718) + Log(-2.54737989184) - Log(-1.6014697

In [4]:
filename = os.path.join("Data","Aula04","MovieReviews.tsv")
#filename = os.path.join("Data","Aula04","Test.tsv")
rawRDD = sc.textFile(filename,2)
header = rawRDD.take(1)[0]

dataRDD = rawRDD.filter(lambda x: x!=header)

weights = [.8, .1, .1]
seed = 42
rawTrainData, rawValidationData, rawTestData = dataRDD.randomSplit(weights, seed)
# Cache the data
rawTrainData.cache()
rawValidationData.cache()
rawTestData.cache()

'''
https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/

The sentiment labels are:

0 - negative
1 - somewhat negative
2 - neutral
3 - somewhat positive
4 - positive
'''

print header
print dataRDD.take(1)


PhraseId	SentenceId	Phrase	Sentiment
[u'1\t1\tA series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .\t1']


In [5]:
numBuckets = 5000
parsedTrainData = rawTrainData.map(lambda x: parseTextPoint(x, numBuckets)).cache()
parsedValData = rawValidationData.map(lambda x: parseTextPoint(x, numBuckets)).cache()
parsedTestData = rawTestData.map(lambda x: parseTextPoint(x, numBuckets)).cache()

binTrainData = parsedTrainData.filter(lambda x: x.label!=2).cache()
binValData = parsedValData.filter(lambda x: x.label!=2).cache()
binTestData = parsedTestData.filter(lambda x: x.label!=2).cache()

print parsedTrainData.take(1)

[LabeledPoint(0.0, (5000,[96,122,143,479,664,1138,1224,1351,1425,1497,1635,1648,1793,2084,2405,2539,2920,3214,3672,3804,3849,3876,4057,4345,4380,4675,4870,4993],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]))]


In [10]:
nb = NaiveBayes()
trainDataNB = nb.Treino(binTrainData)
objet = binValData.take(10)
#print objet[0].features
for i in objet:
    pred = nb.Predicao(i)
    print 'pred ', pred

pred  0.0
pred  0.0
pred  0.0
pred  1.0
pred  1.0
pred  1.0
pred  1.0
pred  1.0
pred  1.0
pred  0.0
